In [463]:
# imports
import requests
import json
import csv
import time
import datetime
from string import Template
from Config import *
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt
import numpy as np

In [464]:
# Variables
headers = {"Authorization": API_TOKEN}
min_stars = 237
max_stars = 10000
last_activity = 90 # within the last __ days
created = 364 * 4 # within the last __ days
min_pull_num = 0 # amount of pull requests a repository needs
watchers = 0 # amount of watchers a repository needs

In [465]:
# Builds the query filter string compatible to github
def query_filter( min_stars, last_activity, created ):
    date_last_act = datetime.datetime.now() - datetime.timedelta( days=last_activity )
    date_created = datetime.datetime.now() - datetime.timedelta( days=created )
    stars = f'{min_stars}..1000'

    return f'is:public archived:false fork:false stars:{stars} pushed:20{date_last_act:%y-%m-%d}..* created:20{date_created:%y-%m-%d}..* sort:stars-asc'

In [466]:
# Funtion that uses requests.post to make the API call
def run_query(query, variables):
    request = requests.post('https://api.github.com/graphql', 
                            json={'query': query, 'variables' : variables}, headers=headers)
    #if request.status_code == 200:
        
    if 'Retry-After' in request.headers:  # reached retry limit
        print(f'[WAITING] for {request.headers["Retry-After"]} seconds before continuing...', end=' ')
        time.sleep(int(request.headers['Retry-After']))

    return request.json()
    #else:
    #    raise Exception(f'ERROR [{request.status_code}]: Query failed to execute...\nRESPONSE: {request.text}')

In [467]:
end_cursor = ""
owner = "astropy"
name = "astropy"
query_string = query_filter( min_stars, last_activity, created)
variables = {
    "owner" : owner,
    "name" : name,
    "end_cursor" : end_cursor,
    "pRsPerPage" : 1 # from 1 to 100
}

In [468]:
# setups the github graphql queries
def setup_query( variables, end_cursor ) :
    variables['end_cursor'] = end_cursor
    query = '''
    query($owner : String!, $name : String!, $pRsPerPage : Int) { 
      repository(owner: $owner, name: $name) {
        id
        name
        pullRequests (first: $pRsPerPage) {
          pageInfo {
            hasNextPage
            endCursor
          }
          nodes {
            author {
                login
            }
            id
            title
            number
            closed
            closedAt
            authorAssociation
            bodyText
            additions
            changedFiles
            deletions
            merged
            mergedAt
            state
            comments(first: 100) {
              edges {
                node {
                  id
                  author {
                    login
                  }
                  authorAssociation
                  bodyText
                  createdAt
                }
              }
            }
            reviewThreads(first:100) {
              edges {
                node {
                  comments(first:100){
                    nodes {
                      id
                      author {
                        login
                      }
                      authorAssociation
                      bodyText
                      createdAt
                    }
                  }
                }
              }
            }
          }
        }
    }
}'''
    query2 = '''
    query($owner : String!, $name : String!, $pRsPerPage : Int, $end_cursor : String!) { 
      repository(owner: $owner, name: $name) {
        id
        name
        pullRequests (first: $pRsPerPage, after: $end_cursor) {
          pageInfo {
            hasNextPage
            endCursor
          }
          nodes {
            author {
                login
            }
            id
            title
            number
            closed
            closedAt
            authorAssociation
            bodyText
            additions
            changedFiles
            deletions
            merged
            mergedAt
            state
            comments(first: 100) {
              edges {
                node {
                  id
                  author {
                    login
                  }
                  authorAssociation
                  bodyText
                  createdAt
                }
              }
            }
            reviewThreads(first:100) {
              edges {
                node {
                  comments(first:100){
                    nodes {
                      id
                      author {
                        login
                      }
                      authorAssociation
                      bodyText
                      createdAt
                    }
                  }
                }
              }
            }
          }
        }
    }
}'''

    if(end_cursor == "") : return (query, variables)
    else : return (query2, variables)

In [469]:
def setup_user_query() :
    query = '''
    query($username : String!) { 
  user(login: $username) {
    id
    pullRequests(first:1) {
      totalCount
    }
    repositories(first:1) {
      totalCount
    }
    repositoriesContributedTo(first:1) {
      totalCount
    }
  }
}'''
    return query

In [470]:
# Runs the query and iterates through all pages of repositories
def iterate_queries( conn, create_dict_method ):

    print("[WORKING] Attemping to add repository ", variables["owner"], variables['name'], 
              "to the database.")
    
    end_cursor = ""
    end_cursor_string = ""
    hasNextPage = True
    index = 0
    
    ai = 1 # slow start: 1, 2, 4, 8 (max)
    md = 0.5
    
    print("[WORKING] Running script to collect all pullrequests. ")
    while( hasNextPage ):
        print("[WORKING] On page " + str(index))
        query = setup_query( variables, end_cursor )
        result = run_query( query[0], query[1] )
        #print(json.dumps(result))
        
        if 'errors' in result:
            if 'timeout' in result['errors'][0]['message']:  # reached timeout
                variables['pRsPerPage'] = int(max(1, variables['pRsPerPage'] * md))  # using AIMD
                ai = 1  # resetting slow start
                print('[WORKING] Timeout! - Reseting page size to : ' + 
                      str(variables['pRsPerPage']))
            if 'maximum' in result['errors'][0]['message']:  # reached timeout
                variables['pRsPerPage'] = int(max(1, variables['pRsPerPage'] * md))  # using AIMD
                ai = 1  # resetting slow start
                print('[WORKING] MAX_NODE_ERROR! - Reseting page size to : ' + 
                      str(variables['pRsPerPage']))
            else:  # some unexpected error.
                print(result['errors'])
                exit(1)
        
        if 'data' in result and result['data']:
            
            #print(json.dumps(result))
                      
            # insert_repositories( conn, result, create_dict_method )
            insert_users_prs_comments( conn, result )
            
            try:
                # if there is a next page, update the endcursor string and continue loop
                if( result["data"]["repository"]["pullRequests"]["pageInfo"]["hasNextPage"] ):
                    end_cursor = result["data"]["repository"]["pullRequests"]["pageInfo"]["endCursor"]
                    
                    variables['pRsPerPage'] = min(100, variables['pRsPerPage'] + ai)  # using AIMD
                    print("[WORKING] New pRsPerPage set to : " + str(variables['pRsPerPage']))
                    ai = min(8, ai * 2)  # slow start
                else:
                    if(result['data']['repository']['pullRequests']['totalCount'] > 1000) :
                        print('[WORKING] We reached the limit of 1,000 repositories.')
                        hasNextPage = False
                        end_cursor = ""
                    else :
                        hasNextPage = False
                        end_cursor = ""
                        print(json.dumps(result))
            except KeyError:
                print("[WORKING] No next page. ")
                break 

            index += 1
        time.sleep(1)

In [471]:
def create_user_tuple( conn, username ) :
    print("[WORKING] Running script to find " + username + " in github. ")
    user_variables = { 'username' : username }
    query = setup_user_query()
    result = run_query( query, user_variables )
    
    if( result['data'] ) :
        if( result['data']['user'] ) :
            user_tuple = (
            result['data']['user']['id'],
            username,
            result['data']['user']['pullRequests']['totalCount'],
            result['data']['user']['repositories']['totalCount'],
            result['data']['user']['repositoriesContributedTo']['totalCount'] )
        else :
            return (
            None,
            username,
            None,
            None,
            None)
    else :
        return (
        None,
        username,
        None,
        None,
        None)
    
    return user_tuple

In [472]:
def create_pr_tuple( node, repo_id, user_id ) :
    return (
    node['id'],
    repo_id,
    user_id,
    node['author']['login'],
    node['number'],
    node['closed'],
    node['authorAssociation'],
    node['bodyText'],
    node['additions'],
    node['deletions'],
    node['changedFiles'],
    node['merged'],
    node['mergedAt'],
    node['state']
    )

In [473]:
def create_comment_tuple( node, pr_id, user_id ) :
    return (
    node['id'],
    pr_id,
    user_id,
    node['author']['login'],
    node['authorAssociation'],
    node['bodyText'],
    node['createdAt']
    )

In [474]:
def find_user_id( username ) :
    sql  = "SELECT id FROM users WHERE users.name=\"{}\"".format( username )
    c = conn.cursor()
    c.execute(sql)
    rows = c.fetchone()
    if rows is not None:
        return rows[0]
    else: 
        user_tuple = create_user_tuple( conn, username )
        user_id = user_tuple[0] # get user id foriegn key
        return( user_tuple, user_id )

In [475]:
# insert repository tuples into the sqlite database
def insert_users_prs_comments( conn, result ):
    # initialize lists
    user_list = []
    pr_list = []
    comment_list = []
    
    # repository id
    repo_id = result['data']['repository']['id']
    
    # insert Pull Request
    pr_index = 0
    pr_nodes = result['data']['repository']['pullRequests']['nodes']
    for pr_node in pr_nodes:
        print("[WORKING] On pull request number: ", pr_index)
        if(pr_node['author']) :
            user_data = find_user_id( pr_node['author']['login'] )
            if isinstance( user_data, str ) : user_id = user_data
            else: 
                print(user_data)
                if(user_data[0]) :
                    user_list.append( user_data[0] )
                user_id = user_data[1]
        else: user_id = None
        
        pr_tuple = create_pr_tuple( pr_node, repo_id, user_id )
        pr_id = pr_tuple[0] # get pr id foriegn key
        pr_list.append( pr_tuple )
        
        # insert Comments
        comm_edges = pr_node['comments']['edges']
        for comm_edge in comm_edges:
            user_data = find_user_id( comm_edge['node']['author']['login'] )
            if isinstance( user_data, str ) : user_id = user_data
            else: 
                if(user_data[0]) :
                    user_list.append( user_data[0] )
                user_id = user_data[1] 
            
            comment_tuple = create_comment_tuple( comm_edge['node'], user_id, pr_id )
            comment_list.append( comment_tuple )
        
        # insert Review Comments
        review_edges = pr_node['reviewThreads']['edges']
        for review_edge in review_edges:
            review_comm_nodes = review_edge['node']['comments']['nodes']
            for review_comm_node in review_comm_nodes:
                user_data = find_user_id( review_comm_node['author']['login'] )
                if isinstance( user_data, str ) : user_id = user_data
                else: 
                    if(user_data[0]) :
                        user_list.append( user_data[0] )
                    user_id = user_data[1] 
                
                comment_tuple = create_comment_tuple( review_comm_node, user_id, pr_id )
                comment_list.append( comment_tuple )
        pr_index+=1
                
    user_sql = """INSERT or IGNORE INTO users (id, name, pr_total, repo_total, repo_contributed_total)
                                            VALUES (?, ?, ?, ?, ?)"""
    pr_sql = """INSERT or IGNORE INTO pull_requests (id, repo_id, user_id, author, number, closed,
                                                     authorAssoc, bodyText, additions, deletions,
                                                     changedFiles, merged, mergedAt, state) VALUES
                                                     (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    comment_sql = """INSERT or IGNORE INTO comments (id, pr_id, user_id, author, authorAssoc,
                                                  bodyText, createdAt) VALUES
                                                 (?, ?, ?, ?, ?, ?, ?)"""
    
    cur = conn.cursor()
    
    print("[WORKING] Attemping to insert users into the database. ")
    cur.executemany(user_sql, user_list)
    conn.commit()
    print("[SUCCESS] Inserted users into the database. ")
    
    print("[WORKING] Attemping to insert pull requests into the database. ")
    cur.executemany(pr_sql, pr_list)
    conn.commit()
    print("[SUCCESS] Inserted pull requests into the database. ")
    
    print("[WORKING] Attemping to insert comments into the database. ")
    cur.executemany(comment_sql, comment_list)
    conn.commit()
    print("[SUCCESS] Inserted commentss into the database. ")

    cur.close()
    return cur.lastrowid

In [476]:
def create_repo_tuple( node ) :
    commits = 0
    if(node["commits"]["target"]["history"]["totalCount"] != None) :
        commits = node["commits"]["target"]["history"]["totalCount"]
        
    primaryLanguage = "null"
    if(node["primaryLanguage"] != None) :
        primaryLanguage = node["primaryLanguage"]["name"]
        
    license_id = "0"
    license_name = "null"
    pseudoLicense = False
    if(node["licenseInfo"] != None) :
        license_id = node["licenseInfo"]["id"]
        license_name = node["licenseInfo"]["name"]
        pseudoLicense = node["licenseInfo"]["pseudoLicense"]
        
    return (
    node["id"], 
    node["name"], 
    node["owner"]["login"], 
    node["createdAt"], 
    node["isMirror"], 
    node["isFork"], 
    node["diskUsage"], 
    primaryLanguage,
    node["contributors"]["totalCount"], 
    node["watchers"]["totalCount"], 
    node["stargazers"]["totalCount"], 
    node["forkCount"], 
    node["issues"]["totalCount"], 
    commits, 
    node["pullRequests"]["totalCount"], 
    node["releases"]["totalCount"],
    license_id, 
    license_name, 
    pseudoLicense, 
    node["url"]
    )

In [477]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

In [478]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [479]:
# insert repository tuples into the sqlite database
def insert_repositories( conn, result, create_dict_method ):
    
    data_list = []
    nodes = result["data"]["search"]["nodes"]
    for node in nodes:
        data_list.append( create_dict_method( node ) )
    
    """
    Create a new repository into the repository table
    :param conn:
    :param repository:
    :return: project id
    """
    sql = """INSERT or IGNORE INTO repositories (id, name, owner, start_date, isMirror, isFork, diskUsage, 
                     primaryLanguage, numContributors, watchers, stars, forks, issues, commits, 
                     pullRequests, releases, license_id, license_name, pseudoLicense, url) VALUES
                     (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    
    cur = conn.cursor()
    cur.executemany(sql, data_list)
    conn.commit()
    print("[SUCCESS] Inserted repos into the database. ")
    cur.close()
    return cur.lastrowid

In [480]:
# name of the sqlite file
database = r".\databases\github_data.db"

# create a database connection
conn = create_connection(database)

In [481]:
# sql for creating the prs and comments tables
sql_create_prs_table = """CREATE TABLE IF NOT EXISTS pull_requests (
                                        id text PRIMARY KEY NOT NULL,
                                        repo_id text FORIEGN KEY NOT NULL,
                                        user_id text FORIEGN KEY,
                                        author text NOT NULL,
                                        number text NOT NULL,
                                        closed boolean NOT NULL,
                                        authorAssoc text NOT NULL,
                                        bodyText text NOT NULL,
                                        additions int NOT NULL,
                                        deletions int NOT NULL,
                                        changedFiles int NOT NULL,
                                        merged boolean NOT NULL,
                                        mergedAt text NOT NULL,
                                        state text NOT NULL
                                    ); """

sql_create_comments_table = """CREATE TABLE IF NOT EXISTS comments (
                                        id text PRIMARY KEY NOT NULL,
                                        pr_id text FORIEGN KEY NOT NULL,
                                        user_id text FORIEGN KEY,
                                        author text NOT NULL,
                                        authorAssoc text NOT NULL,
                                        bodyText text NOT NULL,
                                        createdAt text NOT NULL
                                    ); """

sql_create_users_table = """CREATE TABLE IF NOT EXISTS users (
                                        id text PRIMARY KEY,
                                        name text,
                                        pr_total int,
                                        repo_total int,
                                        repo_contributed_total int
                                    ); """

with conn:
    create_table(conn, sql_create_prs_table)
    create_table(conn, sql_create_comments_table)
    create_table(conn, sql_create_users_table)

In [482]:
sql_grab_repos = """SELECT id, owner, name, primaryLanguage FROM repositories 
                    WHERE primaryLanguage=\"JavaScript\" ORDER BY RANDOM() LIMIT 113;"""

c = conn.cursor()
result = c.execute(sql_grab_repos)
rows = c.fetchall()
rows = np.array(rows)

size = rows.shape[0]

In [483]:
# when conn is valid
with conn:
    # run query to find all repositories
    # pass in the repository tuple builder
    # insert the tuple into the database
    index = 0
    for row in rows:
        
        global variables
        variables['owner'] = row[1]
        variables['name'] = row[2]
        
        print("[UPDATE] On repostory ", index, "of ", size)
    
        iterate_queries( conn, create_repo_tuple )

[UPDATE] On repostory  0 of  113
[WORKING] Attemping to add repository  ethereumbook ethereumbook to the database.
[WORKING] Running script to collect all pullrequests. 
[WORKING] On page 0
[WORKING] On pull request number:  0
[WORKING] Running script to find fjrojasgarcia in github. 
(('MDQ6VXNlcjUxMDQ4MjQ=', 'fjrojasgarcia', 30, 43, 0), 'MDQ6VXNlcjUxMDQ4MjQ=')
[WORKING] Running script to find aantonop in github. 
[WORKING] Running script to find fjrojasgarcia in github. 
[WORKING] Running script to find aantonop in github. 
[WORKING] Running script to find aantonop in github. 
[WORKING] Running script to find aantonop in github. 
[WORKING] Running script to find ryanbrownengineer in github. 
[WORKING] Running script to find aantonop in github. 
[WORKING] Running script to find fjrojasgarcia in github. 
[WORKING] Attemping to insert users into the database. 
[SUCCESS] Inserted users into the database. 
[WORKING] Attemping to insert pull requests into the database. 
[SUCCESS] Inserted 

(('MDQ6VXNlcjI4NzQxOTEw', 'rigzba21', 11, 4, 0), 'MDQ6VXNlcjI4NzQxOTEw')
[WORKING] On pull request number:  10
[WORKING] Running script to find Meshugah in github. 
(('MDQ6VXNlcjU1MzI2NzI=', 'Meshugah', 25, 42, 3), 'MDQ6VXNlcjU1MzI2NzI=')
[WORKING] On pull request number:  11
[WORKING] Running script to find wschwab in github. 
(('MDQ6VXNlcjMxNTkyOTMx', 'wschwab', 27, 38, 7), 'MDQ6VXNlcjMxNTkyOTMx')
[WORKING] On pull request number:  12
[WORKING] Running script to find wschwab in github. 
(('MDQ6VXNlcjMxNTkyOTMx', 'wschwab', 27, 38, 7), 'MDQ6VXNlcjMxNTkyOTMx')
[WORKING] On pull request number:  13
[WORKING] Running script to find haurog in github. 
(('MDQ6VXNlcjM2NTM1Nzc0', 'haurog', 6, 8, 0), 'MDQ6VXNlcjM2NTM1Nzc0')
[WORKING] On pull request number:  14
[WORKING] Running script to find jeamick in github. 
(('MDQ6VXNlcjE5ODk1OTM1', 'jeamick', 44, 104, 8), 'MDQ6VXNlcjE5ODk1OTM1')
[WORKING] On pull request number:  15
[WORKING] Running script to find rigzba21 in github. 
(('MDQ6VXNlcjI4N

(('MDQ6VXNlcjY1NTcyNTA=', 'subtly', 141, 25, 3), 'MDQ6VXNlcjY1NTcyNTA=')
[WORKING] On pull request number:  9
[WORKING] On pull request number:  10
[WORKING] On pull request number:  11
[WORKING] Running script to find jaycenhorton in github. 
(('MDQ6VXNlcjE4NDA3MDEz', 'jaycenhorton', 136, 39, 10), 'MDQ6VXNlcjE4NDA3MDEz')
[WORKING] Running script to find stefek99 in github. 
[WORKING] On pull request number:  12
[WORKING] On pull request number:  13
[WORKING] On pull request number:  14
[WORKING] On pull request number:  15
[WORKING] On pull request number:  16
[WORKING] On pull request number:  17
[WORKING] On pull request number:  18
[WORKING] On pull request number:  19
[WORKING] Running script to find anticlimactic in github. 
(('MDQ6VXNlcjk1Njg3NTY=', 'anticlimactic', 9, 5, 3), 'MDQ6VXNlcjk1Njg3NTY=')
[WORKING] On pull request number:  20
[WORKING] Running script to find yuric in github. 
(('MDQ6VXNlcjEwNTIyNTk=', 'yuric', 38, 51, 0), 'MDQ6VXNlcjEwNTIyNTk=')
[WORKING] On pull requ

(('MDQ6VXNlcjI1MzU3NDk3', 'ohadh123', 10, 6, 0), 'MDQ6VXNlcjI1MzU3NDk3')
[WORKING] On pull request number:  35
[WORKING] Running script to find adroitperson37 in github. 
(('MDQ6VXNlcjE3NDEzOTA1', 'adroitperson37', 4, 13, 0), 'MDQ6VXNlcjE3NDEzOTA1')
[WORKING] On pull request number:  36
[WORKING] On pull request number:  37
[WORKING] On pull request number:  38
[WORKING] On pull request number:  39
[WORKING] On pull request number:  40
[WORKING] On pull request number:  41
[WORKING] On pull request number:  42
[WORKING] On pull request number:  43
[WORKING] Running script to find AyushyaChitransh in github. 
(('MDQ6VXNlcjEzOTE0NjM0', 'AyushyaChitransh', 41, 47, 7), 'MDQ6VXNlcjEzOTE0NjM0')
[WORKING] On pull request number:  44
[WORKING] Running script to find sgvictorino in github. 
(('MDQ6VXNlcjkxNzAzMTY=', 'sgvictorino', 57, 82, 3), 'MDQ6VXNlcjkxNzAzMTY=')
[WORKING] Running script to find sgvictorino in github. 
[WORKING] On pull request number:  45
[WORKING] Running script to find Ay

(('MDQ6VXNlcjEwMDg2OTIy', 'daveai', 5, 6, 2), 'MDQ6VXNlcjEwMDg2OTIy')
[WORKING] On pull request number:  23
[WORKING] On pull request number:  24
[WORKING] On pull request number:  25
[WORKING] On pull request number:  26
[WORKING] Running script to find abhiShandy in github. 
(('MDQ6VXNlcjY5OTg1OTM=', 'abhiShandy', 32, 22, 5), 'MDQ6VXNlcjY5OTg1OTM=')
[WORKING] On pull request number:  27
[WORKING] Running script to find zasnicoff in github. 
(('MDQ6VXNlcjY4MjM2NTY=', 'zasnicoff', 7, 3, 0), 'MDQ6VXNlcjY4MjM2NTY=')
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] Running script to find zasnicoff in github. 
[WORKING] On pull request number:  28
[WORKING] Running script to find abhiShandy in github. 
(

...

(('MDQ6VXNlcjQ4MzY0', 'splix', 277, 55, 26), 'MDQ6VXNlcjQ4MzY0')
[WORKING] On pull request number:  20
[WORKING] On pull request number:  21
[WORKING] Running script to find Orlandao in github. 
(('MDQ6VXNlcjM0ODM0MjEw', 'Orlandao', 2, 1, 0), 'MDQ6VXNlcjM0ODM0MjEw')
[WORKING] On pull request number:  22
[WORKING] Running script to find Orlandao in github. 
(('MDQ6VXNlcjM0ODM0MjEw', 'Orlandao', 2, 1, 0), 'MDQ6VXNlcjM0ODM0MjEw')
[WORKING] On pull request number:  23
[WORKING] Running script to find FlashSheridan in github. 
(('MDQ6VXNlcjc1ODE5OQ==', 'FlashSheridan', 23, 14, 8), 'MDQ6VXNlcjc1ODE5OQ==')
[WORKING] On pull request number:  24
[WORKING] Running script to find dcheli in github. 
(('MDQ6VXNlcjI3NzI4OTQ=', 'dcheli', 5, 7, 1), 'MDQ6VXNlcjI3NzI4OTQ=')
[WORKING] On pull request number:  25
[WORKING] Running script to find rjpeterson in github. 
(('MDQ6VXNlcjQwOTU4NTI=', 'rjpeterson', 5, 15, 2), 'MDQ6VXNlcjQwOTU4NTI=')
[WORKING] Attemping to insert users into the database. 
[SUCCESS

(('MDQ6VXNlcjM1NDQ=', 'gene-smith', 9, 69, 1), 'MDQ6VXNlcjM1NDQ=')
[WORKING] Running script to find corysolovewicz in github. 
[WORKING] Running script to find gene-smith in github. 
[WORKING] On pull request number:  31
[WORKING] Attemping to insert users into the database. 
[SUCCESS] Inserted users into the database. 
[WORKING] Attemping to insert pull requests into the database. 
[SUCCESS] Inserted pull requests into the database. 
[WORKING] Attemping to insert comments into the database. 
[SUCCESS] Inserted commentss into the database. 
[WORKING] New pRsPerPage set to : 40
[WORKING] On page 18
[WORKING] On pull request number:  0
[WORKING] On pull request number:  1
[WORKING] On pull request number:  2
[WORKING] Running script to find jiangleo in github. 
(('MDQ6VXNlcjE1MTM0ODk2', 'jiangleo', 21, 31, 0), 'MDQ6VXNlcjE1MTM0ODk2')
[WORKING] On pull request number:  3
[WORKING] Running script to find tim-hub in github. 
(('MDQ6VXNlcjI4ODQzMjA=', 'tim-hub', 89, 177, 19), 'MDQ6VXNlcjI4OD

[WORKING] On page 20
[WORKING] MAX_NODE_ERROR! - Reseting page size to : 28
[WORKING] On page 20
[WORKING] On pull request number:  0
[WORKING] On pull request number:  1
[WORKING] On pull request number:  2
[WORKING] On pull request number:  3
[WORKING] On pull request number:  4
[WORKING] On pull request number:  5
[WORKING] On pull request number:  6
[WORKING] On pull request number:  7
[WORKING] On pull request number:  8
[WORKING] Running script to find auino in github. 
(('MDQ6VXNlcjcyNjU4NjA=', 'auino', 4, 29, 3), 'MDQ6VXNlcjcyNjU4NjA=')
[WORKING] On pull request number:  9
[WORKING] Running script to find auino in github. 
(('MDQ6VXNlcjcyNjU4NjA=', 'auino', 4, 29, 3), 'MDQ6VXNlcjcyNjU4NjA=')
[WORKING] On pull request number:  10
[WORKING] Running script to find auino in github. 
(('MDQ6VXNlcjcyNjU4NjA=', 'auino', 4, 29, 3), 'MDQ6VXNlcjcyNjU4NjA=')
[WORKING] On pull request number:  11
[WORKING] On pull request number:  12
[WORKING] On pull request number:  13
[WORKING] On pull 

(('MDQ6VXNlcjYyOTQwNDQ=', 'karolinkas', 23, 73, 3), 'MDQ6VXNlcjYyOTQwNDQ=')
[WORKING] Running script to find karolinkas in github. 
[WORKING] On pull request number:  24
[WORKING] Running script to find Jawknee-88 in github. 
(('MDQ6VXNlcjMzODA5Mjg3', 'Jawknee-88', 1, 8, 0), 'MDQ6VXNlcjMzODA5Mjg3')
[WORKING] Running script to find Jawknee-88 in github. 
[WORKING] On pull request number:  25
[WORKING] On pull request number:  26
[WORKING] On pull request number:  27
[WORKING] Running script to find rluijk in github. 
(('MDQ6VXNlcjQyNDcyOQ==', 'rluijk', 3, 21, 0), 'MDQ6VXNlcjQyNDcyOQ==')
[WORKING] On pull request number:  28
[WORKING] Running script to find zabaladev in github. 
(('MDQ6VXNlcjk0MjIzNTg=', 'zabaladev', 1, 1, 0), 'MDQ6VXNlcjk0MjIzNTg=')
[WORKING] On pull request number:  29
[WORKING] Running script to find VieYang in github. 
(('MDQ6VXNlcjExNjc2ODI4', 'VieYang', 9, 92, 5), 'MDQ6VXNlcjExNjc2ODI4')
[WORKING] On pull request number:  30
[WORKING] Running script to find brianmc

TypeError: 'NoneType' object is not subscriptable